In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# $\theta_0$ for different components
For foils, $\theta_0$ is fixed by the need for the foil to rotate the given $\lambda_0$ by $\pi$. 

In wollaston prisms and isosceles triangles $\theta_0$ is just a function of the geometry.

For a triangle with base 0.6, height 0.15, the angle is $\arctan(0.15/(0.6/2)) = 0.4636476090008061$. The WSP has the same angle, having just one inclined plane and a base twice as great as the height. 

In [ ]:
from definitions import c

def tune_foil(lambda_0, B_s, d, c, n = 0):
    return np.arcsin(c * d * B_s * lambda_0 / ((2 * n + 1) * np.pi))

B_s = 1
d = 3e-6
# theta_0 = 0.0960
for lambda_0 in [2.165e-10, 3e-10, 4.321e-10, 8e-10, 10e-10]:
    theta_0 = tune_foil(lambda_0, B_s, d, c, n = 0)
    theta_0_deg = np.rad2deg(theta_0)
    phi_foil = c * d * B_s * lambda_0 / np.sin(theta_0)
    print(theta_0, theta_0_deg, phi_foil)

In [ ]:
np.arctan(0.15/0.3)

# $\delta_y$ range for foils
Foils have a set $\theta_0$ as well as a minimal field strength of 0.3mT, giving a natural floor to $\delta_y$ given the other parameters $L_s, \lambda_0$.

In [ ]:
for lambda_0 in [2.165e-10, 4.321e-10, 8e-10]:
    theta_0 = tune_foil(lambda_0, B_s, d, c)
    theta_0_deg = np.rad2deg(theta_0)
    phi_foil = c * d * B_s * lambda_0 / np.sin(theta_0)
    print(theta_0, theta_0_deg, phi_foil)
    L_s = 1.8
    delta_y_min = c * lambda_0 ** 2 * L_s * (0.3e-3) / (np.pi * np.tan(theta_0)) 
    print(delta_y_min)

# Small-angle error correction factor


$\delta = f_0 L_s \lambda_0$
In small-angle approximation
$$\phi_t = \delta Q$$
With error, an additional factor is incurred
$$\phi_t = \delta \frac{1}{1 - \frac{y^2}{2L_s^2}}Q = f_0 L_s \lambda_0 \frac{1}{1 - \frac{y^2}{2L_s^2}}Q$$
This means that the direct correspondence between the modulation phase $\phi_t$ and scattering at characteristic length $\delta$ breaks down. In practice this correction can be used to bound $y$ for which the approximation is valid.

In [ ]:
from definitions import *

h_d = detector_size
y = np.linspace(-h_d/2, h_d/2, 100000)
L_s = 1.5
f_0 = 1e5
f = 1 / (1 - y ** 2 / (2 * L_s ** 2))
I_base = np.cos(2 * np.pi * f_0 * y)
I_corrected = np.cos(2 * np.pi * f_0 * y * f)
# plt.plot(y, f)
plt.plot(y, I_base, '.')
plt.plot(y, I_corrected, '.')
plt.xlim((-detector_size/2, -detector_size/2 +1e-5))

In [ ]:
from scipy.signal import hilbert

f_s = 1/detector_pixel_size
f_super_sampled = f_s / 2

for f_0 in [1e2, 1e3, 1e4, f_super_sampled]:
    for L_s in [0.5, 0.75, 1.0, 1, 1.5, 2.0]:
        # L_s = 1.5
        # f_0 = 1e5
        f = 1 / (1 - y ** 2 / (2 * L_s ** 2))
        I_base = np.cos(2 * np.pi * f_0 * y)
        I_corrected = np.cos(2 * np.pi * f_0 * y * f)
        I_error = I_corrected - I_base
        # plt.plot(y, I_error)
        analytic_signal = hilbert(I_error)
        envelope = np.abs(analytic_signal)
        plt.plot(y,envelope, label=rf'$L_s = {L_s}$m')
    plt.grid()
    plt.title(rf"$f_0 = {f_0}$")
    plt.legend()
    plt.xlim((-detector_size*0.499, detector_size*0.499))
    plt.show()
